In [1]:
import data.data_source as data_source
import time
from domino.data_sources import DataSourceClient
from datetime import date
from dateutil.relativedelta import relativedelta
import pandas as pd
from io import StringIO
from functools import lru_cache
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from config import env
from utils.artifact_saver import get_artifact_path

import mlflow
import os

experiment_name = f"Populate Tsy Curve [{env}]"
mlflow.set_experiment(experiment_name)

# ─── Helpers ────────────────────────────────────────────────────────────────

@lru_cache(maxsize=100)
def fetch_treasury_csv(year: int) -> str:
    url = (
        f"https://home.treasury.gov/resource-center/data-chart-center/interest-rates/"
        f"daily-treasury-rates.csv/{year}/all"
        f"?field_tdr_date_value={year}"
        f"&type=daily_treasury_yield_curve&page&_format=csv"
    )
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.text

def parse_tenor(tenor_str: str) -> float:
    num_str, unit = tenor_str.strip().split(maxsplit=1)
    n = float(num_str); u = unit.lower()
    if u.startswith('mo'):   return (n * 30) / 360
    if u.startswith('yr'):   return n
    if u.startswith('day'):  return n / 360
    raise ValueError(f"Unknown tenor unit: '{unit}'")

def prepare_year_rows(year, start_date, end_date):
    """
    Fetch & parse a single year's CSV, return a flat list of rows:
    (curve_type, date, tenor, rate, tenor_num)
    """
    text = fetch_treasury_csv(year)
    df   = pd.read_csv(StringIO(text), parse_dates=['Date'], index_col='Date')
    df = df[(df.index.date >= start_date) & (df.index.date <= end_date)]
    rows = []
    for ts, row in df.iterrows():
        d = ts.date()
        for tenor, rate in row.items():
            if pd.isna(rate):
                continue
            rows.append((
                'US Treasury Par',
                d.isoformat(),
                tenor,
                float(rate),
                parse_tenor(tenor)
            ))
    return year, rows

# ─── Main loader ────────────────────────────────────────────────────────────


def populate(
    days: int,
    batch_size: int    = 5000,
    fetch_workers: int = 4,
    write_workers: int = 2
):
    
    """
    Populate rate_curves for the last `days` days (up to today),
    but not before 2010-03-15.
    """
    # calculate date range
    with mlflow.start_run():
        mlflow.log_param("days_requested", days)
        mlflow.log_param("starting_domino_user", os.environ["DOMINO_STARTING_USERNAME"])
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("fetch_workers", fetch_workers)
        mlflow.log_param("write_workers", write_workers)

        start_time = time.time()
        end_date = date.today()
        start_date = end_date - relativedelta(days=days)
        min_date = date(2010, 3, 15)
        unique_dates = set()
        if start_date < min_date:
            start_date = min_date
    
        years = list(range(start_date.year, end_date.year + 1))
    
        # 1) parallel fetch + parse per-year
        rows_by_year = {}
        with ThreadPoolExecutor(max_workers=fetch_workers) as fetch_pool:
            futures = {
                fetch_pool.submit(prepare_year_rows, y, start_date, end_date): y
                for y in years
            }
            for fut in as_completed(futures):
                y = futures[fut]
                try:
                    year, rows = fut.result()
                    if rows:
                        rows_by_year[year] = rows
                        print(f"{year}: prepared {len(rows)} rows")
                    else:
                        print(f"{year}: no data → skipped")
                except Exception as e:
                    print(f"{y}: error fetching/parsing → {e}")

        ds = data_source.get_data_source()

        # 2) for each year, batch & fire INSERTs in parallel
        def write_batch(batch):
            unique_dates.update([r[1] for r in batch])
            vals = ", ".join(
                f"('{r[0]}','{r[1]}','{r[2]}',{r[3]},{r[4]})"
                for r in batch
            )
            sql = f"""
                INSERT INTO rate_curves
                  (curve_type, curve_date, tenor_str, rate, tenor_num)
                VALUES
                  {vals}
                ON CONFLICT (curve_type, curve_date, tenor_str) DO UPDATE
                  SET rate      = EXCLUDED.rate,
                      tenor_num = EXCLUDED.tenor_num;
                """
            ds.query(sql)
            
        with ThreadPoolExecutor(max_workers=write_workers) as write_pool:
            write_futures = []
            for year, rows in rows_by_year.items():
                for i in range(0, len(rows), batch_size):
                    batch = rows[i : i + batch_size]
                    write_futures.append(write_pool.submit(write_batch, batch))
    
            for fut in as_completed(write_futures):
                try:
                    fut.result()
                except Exception as e:
                    print(f"Write error: {e}")


        duration = time.time() - start_time
        num_rows  = sum(len(r) for r in rows_by_year.values())

        # log metrics
        mlflow.log_metric("days_loaded", len(unique_dates))
        mlflow.log_metric("rows_loaded", num_rows)
        mlflow.log_metric("duration_seconds", duration)

        # artifact: snapshot all rows as CSV
        all_rows = [r for rows in rows_by_year.values() for r in rows]
        df_all = pd.DataFrame(all_rows, columns=[
            "curve_type", "curve_date", "tenor_str", "rate", "tenor_num"
        ])
        csv_path = get_artifact_path("rate_curves_loaded.csv")
        df_all.to_csv(csv_path, index=False)
        mlflow.log_artifact(csv_path, artifact_path="rate_curves")
        
        print("✅ Done bulk-loading rate_curves "
              f"from {start_date} through {end_date}")

# ─── MAIN ───────────────────────────────────────────────────────────────────
# arg1 is the number of days to backdate.
# 1 => yesterday's curve, 100 => last 100 days.
default_backdated_days = 10

if __name__ == '__main__':
    d = default_backdated_days
else:
    try:
        days_to_backdate = sys.argv[1]
        d = int(days_to_backdate)
    except Exception as e:
        d = default_backdated_days

populate(days=d)    
    

'data_env' not found in environment. Defaulting to 'sandbox' env.
setting env to sandbox data
2025: prepared 98 rows
getting data source for sandbox
✅ Done bulk-loading rate_curves from 2025-05-31 through 2025-06-10
🏃 View run abundant-mouse-996 at: http://127.0.0.1:8768/#/experiments/1500/runs/0083c4ae5ab74db3a46c554799efc1cb
🧪 View experiment at: http://127.0.0.1:8768/#/experiments/1500
